In [1]:
import io, os, sys, types, subprocess, collections

# Import Pandas
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import Plotly and Cufflinks
# Plotly username and API key should be set in environment variables
import plotly
plotly.tools.set_credentials_file(username=os.environ['PLOTLY_USERNAME'], api_key=os.environ['PLOTLY_KEY'])
import plotly.graph_objs as go
import cufflinks as cf

# Import numpy
import numpy as np

# Import SciKit decision tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# Import Scikit cross-validation function
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

In [2]:
# Import module to read in secure data
sys.path.append('../data/NREL')
import retrieve_data as rd

In [3]:
solar = rd.retrieve_dirks_sheet()

In [6]:
sys.path.append('utils')
import process_data as prd

In [7]:
prd.clean_data(solar)

### Graphs - Heatmap of relationships between degradation modes
#### Variation A: Multiply data entries for module systems by number of modules
Number of modules = number of single modules + (number of systems * modules per system)

In [9]:
def get_weighted_mode_percentage(df, mode):
    """
    Return the percentage of modules with a reported degradation mode
    NOTE: This function will multiply systems of modules by the number of modules it holds
    
    Args:
        df (DataFrame): Pandas DataFrame that has been cleaned using the clean_data function
        mode (string): Degradation mode to find in the DataFrame
    Returns:
        float: The percentage of modules with specified degradation mode
    """
    single_modules = df[df['System/module'] == 'Module']
    mult_modules = df[df['System/module'] != 'Module']
    total_modules = single_modules.shape[0] + mult_modules['No.modules'].sum()

    if total_modules == 0:
        return 0
    
    mode_single_modules = single_modules[single_modules['Cause (Cleaned)'].notnull()]
    mode_single_modules = mode_single_modules[mode_single_modules['Cause (Cleaned)'].apply(lambda x: any(pd.Series(x).str.contains(mode)))]
    num_mode_single_modules = mode_single_modules.shape[0]
    
    mode_mult_modules = mult_modules[mult_modules['Cause (Cleaned)'].notnull()]
    mode_mult_modules = mode_mult_modules[mode_mult_modules['Cause (Cleaned)'].apply(lambda x: any(pd.Series(x).str.contains(mode)))]
    if mode_mult_modules.empty:
        num_mode_mult_modules = 0
    else:
        num_mode_mult_modules = mode_mult_modules['No.modules'].sum()
    
    return float(num_mode_single_modules + num_mode_mult_modules) / total_modules

In [10]:
def generate_weighted_heatmap(df, modes):
    """
    Return the percentage of modules with a reported degradation mode
    NOTE: This function will multiply systems of modules by the number of modules it holds
    
    Args:
        df (DataFrame): Pandas DataFrame that has been cleaned using the clean_data function
        mode (string): Degradation mode to find in the DataFrame
    Returns:
        float: The percentage of modules with specified degradation mode
    """
    heatmap = pd.DataFrame(data=None, columns=modes, index=modes)
    
    # Calculate total number of modules
    single_modules = df[df['System/module'] == 'Module']
    mult_modules = df[df['System/module'] != 'Module']

    num_single_modules = single_modules.shape[0]
    num_mult_modules = mult_modules['No.modules'].sum()
    total_modules = num_single_modules + num_mult_modules
    
    mode_percentages = {}
    for mode in modes:
        mode_percentages[mode] = get_weighted_mode_percentage(df, mode)
    
    for mode_1 in modes:
        for mode_2 in modes:
            if mode_1 == mode_2:
                heatmap.set_value(mode_1, mode_2, np.nan)
            else:
                print(mode_1 + " & " + mode_2)
                
                heatmap_reflection = heatmap.at[mode_2, mode_1]
                if (not pd.isnull(heatmap_reflection)):
                    heatmap.set_value(mode_1, mode_2, heatmap_reflection)
                    continue
                
                percentage_1 = mode_percentages[mode_1]
                percentage_2 = mode_percentages[mode_2]
                
                print('Percentage 1: ' + str(percentage_1))
                print('Percentage 2: ' + str(percentage_2))
                
                if (percentage_1 == 0 or percentage_2 == 0):
                    heatmap.set_value(mode_1, mode_2, 0)
                    continue

                mode_single_modules_both = single_modules[single_modules['Cause (Cleaned)'].notnull()]
                mode_single_modules_both = mode_single_modules_both[mode_single_modules_both['Cause (Cleaned)'].apply(lambda x: any(pd.Series(x).str.contains(mode_1))
                                                                                                 and any(pd.Series(x).str.contains(mode_2)))]
                num_mode_single_modules_both = mode_single_modules_both['Cause (Cleaned)'].shape[0]

                mode_single_modules_both = mult_modules[mult_modules['Cause (Cleaned)'].notnull()]
                mode_mult_modules_both = mode_single_modules_both[mode_single_modules_both['Cause (Cleaned)'].apply(lambda x: any(pd.Series(x).str.contains(mode_1))
                                                                                           and any(pd.Series(x).str.contains(mode_2)))]
                num_mode_mult_modules_both = mode_mult_modules_both['No.modules'].sum()

                percentage_both = (num_mode_single_modules_both + num_mode_mult_modules_both) / total_modules
                print('Percentage Both: ' + str(percentage_both))

                result = percentage_both / (percentage_1 * percentage_2)
                print('Result: ' + str(result))

                heatmap.set_value(mode_1, mode_2, result)
                
    return heatmap

#### Generate heatmap for the entire dataset, regardless of time

In [11]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[(solar['System/module'] != 'System') | (solar['No.modules'].notnull())]

sys_heatmap_all = generate_weighted_heatmap(specified, modes)

Encapsulant discoloration & Major delamination
Percentage 1: 0.236915056704
Percentage 2: 0.888442213156
Percentage Both: 0.230479788535
Result: 1.09499213333
Encapsulant discoloration & Minor delamination
Percentage 1: 0.236915056704
Percentage 2: 8.49843266806e-08
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.236915056704
Percentage 2: 0.00813317003198
Percentage Both: 0.00813317003198
Result: 4.22092210564
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.236915056704
Percentage 2: 0.467244592949
Percentage Both: 0.0100421729622
Result: 0.0907174325493
Encapsulant discoloration & Hot spots
Percentage 1: 0.236915056704
Percentage 2: 0.68649463754
Percentage Both: 0.228753671875
Result: 1.40649522599
Encapsulant discoloration & Fractured cells
Percentage 1: 0.236915056704
Percentage 2: 0.00278476641667
Percentage Both: 0.00264870650965
Result: 4.01469358113
Encapsulant discoloration & Diode/J-box problem
Percent

In [12]:
sys_heatmap_all

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.09499,0,4.22092,0.0907174,1.4065,4.01469,4.19139,3.98584,3.90533,4.1618
Major delamination,1.09499,NaN,0,0.978916,1.11964,1.1218,1.12553,1.0638,0.858912,1.04128,1.12166
Minor delamination,0,0,NaN,0,0,0,0,0,73.992,0,0
Backsheet other,4.22092,0.978916,0,NaN,1.86136,1.45661,0,18.5057,64.3516,0,0
Internal circuitry discoloration,0.0907174,1.11964,0,1.86136,NaN,1.44681,0.104502,0.441673,1.58939,0.000185497,0.023004
Hot spots,1.4065,1.1218,0,1.45661,1.44681,NaN,1.44814,1.37028,0.84153,0,1.45162
Fractured cells,4.01469,1.12553,0,0,0.104502,1.44814,NaN,0,0,0.0103746,0
Diode/J-box problem,4.19139,1.0638,0,18.5057,0.441673,1.37028,0,NaN,14.9734,0,0
Glass breakage,3.98584,0.858912,73.992,64.3516,1.58939,0.84153,0,14.9734,NaN,68.4511,0.00787428
Permanent soiling,3.90533,1.04128,0,0,0.000185497,0,0.0103746,0,68.4511,NaN,0.527851


In [13]:
sys_heatmap_all.iplot(kind='heatmap',colorscale='spectral',
                                filename='sys-heatmap-all', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed before 2000

In [11]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[(solar['System/module'] != 'System') | (solar['No.modules'].notnull())]
specified = specified[specified['Begin.Year'] < 2000]

sys_heatmap_pre_2000 = generate_weighted_heatmap(specified, modes)

Encapsulant discoloration & Major delamination
Percentage 1: 0.067517053776
Percentage 2: 0.946678967346
Percentage Both: 0.0636983482767
Result: 0.996579524123
Encapsulant discoloration & Minor delamination
Percentage 1: 0.067517053776
Percentage 2: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.067517053776
Percentage 2: 0.00951236408476
Percentage Both: 0.00951236408476
Result: 14.811072819
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.067517053776
Percentage 2: 0.626507928105
Percentage Both: 0.013503753609
Result: 0.319237904359
Encapsulant discoloration & Hot spots
Percentage 1: 0.067517053776
Percentage 2: 0.672691704867
Percentage Both: 0.0599501563733
Result: 1.31995998335
Encapsulant discoloration & Fractured cells
Percentage 1: 0.067517053776
Percentage 2: 2.18283578849e-05
Percentage Both: 2.18283578849e-05
Result: 14.811072819
Encapsulant discoloration & Diode/J-box problem
Percentage 1: 0.067517053776
Percentage 2: 0.0631983

In [12]:
sys_heatmap_pre_2000

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,0.99658,0,14.8111,0.319238,1.31996,14.8111,14.7075,14.8056,13.8554,0.95002
Major delamination,0.99658,NaN,0,1.0563,1.05075,1.05555,1.05632,0.998371,0.851148,0.988131,0
Minor delamination,0,0,NaN,0,0,0,0,0,0,0,0
Backsheet other,14.8111,1.0563,0,NaN,1.59613,1.48653,0,15.8228,58.8634,0,0
Internal circuitry discoloration,0.319238,1.05075,0,1.59613,NaN,1.47647,0,0.329389,1.26812,0,0.102381
Hot spots,1.31996,1.05555,0,1.48653,1.47647,NaN,0,1.39842,0.832362,0,0
Fractured cells,14.8111,1.05632,0,0,0,0,NaN,0,0,0,0
Diode/J-box problem,14.7075,0.998371,0,15.8228,0.329389,1.39842,0,NaN,11.9121,0,0
Glass breakage,14.8056,0.851148,0,58.8634,1.26812,0.832362,0,11.9121,NaN,55.064,3.77569
Permanent soiling,13.8554,0.988131,0,0,0,0,0,0,55.064,NaN,251.496


In [13]:
sys_heatmap_pre_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='sys-heatmap-pre-2000', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed post 2000

In [14]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[(solar['System/module'] != 'System') | (solar['No.modules'].notnull())]
specified = specified[specified['Begin.Year'] >= 2000]

sys_heatmap_post_2000 = generate_weighted_heatmap(specified, modes)

Encapsulant discoloration & Major delamination
Percentage 1: 0.728248540003
Percentage 2: 0.719529658385
Percentage Both: 0.714224011912
Result: 1.36303222747
Encapsulant discoloration & Minor delamination
Percentage 1: 0.728248540003
Percentage 2: 3.31478600073e-07
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.728248540003
Percentage 2: 0.00413287518571
Percentage Both: 0.00413287518571
Result: 1.37315757612
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.728248540003
Percentage 2: 0.00530730386577
Percentage Both: 1.98887160044e-06
Result: 0.000514580317076
Encapsulant discoloration & Hot spots
Percentage 1: 0.728248540003
Percentage 2: 0.726530486418
Percentage Both: 0.718362853713
Result: 1.35772058216
Encapsulant discoloration & Fractured cells
Percentage 1: 0.728248540003
Percentage 2: 0.0107985783546
Percentage Both: 0.0102678811159
Result: 1.30567360646
Encapsulant discoloration & Diode/J-box problem
Pe

In [15]:
sys_heatmap_post_2000

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.36303,0,1.37316,0.00051458,1.35772,1.30567,1.20151,0.208752,1.04912,1.35603
Major delamination,1.36303,NaN,0,0.000111469,1.38893,1.3754,1.38975,0.173725,0.211281,1.05982,1.38728
Minor delamination,0,0,NaN,0,0,0,0,0,290.634,0,0
Backsheet other,1.37316,0.000111469,0,NaN,0,1.37618,0,0,0,0,0
Internal circuitry discoloration,0.00051458,1.38893,0,0,NaN,1.37546,9.25412,94.2098,0,0.273336,2.00851
Hot spots,1.35772,1.3754,0,1.37618,1.37546,NaN,1.37636,0,1.14568,0,1.37391
Fractured cells,1.30567,1.38975,0,0,9.25412,1.37636,NaN,0,0,0.0447799,0
Diode/J-box problem,1.20151,0.173725,0,0,94.2098,0,0,NaN,36.3293,0,0
Glass breakage,0.208752,0.211281,290.634,0,0,1.14568,0,36.3293,NaN,221.63,0
Permanent soiling,1.04912,1.05982,0,0,0.273336,0,0.0447799,0,221.63,NaN,0.00107989


In [16]:
sys_heatmap_post_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='sys-heatmap-post-2000', margin=(200,150,120,30))

#### Variation B: Count module systems as single module
Number of modules = number of single modules + number of systems

In [15]:
def get_unweighted_mode_percentage(df, mode):
    """
    Return the percentage of modules with a reported degradation mode
    
    Args:
        df (DataFrame): Pandas DataFrame that has been cleaned using the clean_data function
        mode (string): Degradation mode to find in the DataFrame
    Returns:
        float: The percentage of modules with specified degradation mode
    """
    # Calculate total number of modules
    total_modules = df.shape[0]
    if total_modules == 0:
        return 0
    
    mode_modules = df[df['Cause (Cleaned)'].notnull()]
    mode_modules = mode_modules[mode_modules['Cause (Cleaned)'].apply(lambda x: any(pd.Series(x).str.contains(mode)))]
    if mode_modules.empty:
        num_mode_modules = 0
    else:
        num_mode_modules = mode_modules.shape[0]
    
    return float(num_mode_modules) / total_modules

In [16]:
def generate_unweighted_heatmap(df, modes):
    """
    Returns a DataFrame used to construct a heatmap based on frequency of two degradation modes appearing together
    
    Args:
        df (DataFrame): A *cleaned* DataFrame containing the data entries to check modes from
        modes (List of String): A list of all modes to check for in the DataFrame
    
    Returns:
        heatmap (DataFrame): DataFrame containing all of degradation modes correlation frequency results
    """
    heatmap = pd.DataFrame(data=None, columns=modes, index=modes)
    
    # Calculate total number of modules
    total_modules = df.shape[0]

    mode_percentages = {}
    for mode in modes:
        mode_percentages[mode] = get_unweighted_mode_percentage(df, mode)
    
    for mode_1 in modes:
        for mode_2 in modes:
            if mode_1 == mode_2:
                heatmap.set_value(mode_1, mode_2, np.nan)
            else:
                print(mode_1 + " & " + mode_2)
                
                # 
                heatmap_reflection = heatmap.at[mode_2, mode_1]
                if (not pd.isnull(heatmap_reflection)):
                    heatmap.set_value(mode_1, mode_2, heatmap_reflection)
                    continue
                
                percentage_1 = mode_percentages[mode_1]
                percentage_2 = mode_percentages[mode_2]
                
                print('Percentage 1: ' + str(percentage_1))
                print('Percentage 2: ' + str(percentage_2))
                
                if (percentage_1 == 0 or percentage_2 == 0):
                    heatmap.set_value(mode_1, mode_2, 0)
                    continue

                mode_single_modules_both = df[df['Cause (Cleaned)'].notnull()]
                mode_single_modules_both = mode_single_modules_both[mode_single_modules_both['Cause (Cleaned)'].apply(lambda x: any(pd.Series(x).str.contains(mode_1))
                                                                                       and any(pd.Series(x).str.contains(mode_2)))]
                num_mode_single_modules_both = mode_single_modules_both['Cause (Cleaned)'].shape[0]

                percentage_both = float(num_mode_single_modules_both) / total_modules
                print('Percentage Both: ' + str(percentage_both))

                result = float(percentage_both) / (percentage_1 * percentage_2)
                print('Result: ' + str(result))

                heatmap.set_value(mode_1, mode_2, result)
                
    return heatmap

In [17]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

modes_heatmap_all = generate_unweighted_heatmap(solar, modes)

Encapsulant discoloration & Major delamination
Percentage 1: 0.336320840123
Percentage 2: 0.649194278472
Percentage Both: 0.282998370451
Result: 1.29615062012
Encapsulant discoloration & Minor delamination
Percentage 1: 0.336320840123
Percentage 2: 9.05305087815e-05
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.336320840123
Percentage 2: 0.0307803729857
Percentage Both: 0.0307803729857
Result: 2.9733512786
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.336320840123
Percentage 2: 0.265707043274
Percentage Both: 0.0342205323194
Result: 0.382939278811
Encapsulant discoloration & Hot spots
Percentage 1: 0.336320840123
Percentage 2: 0.446858591345
Percentage Both: 0.218721709216
Result: 1.45535184139
Encapsulant discoloration & Fractured cells
Percentage 1: 0.336320840123
Percentage 2: 0.0370269780916
Percentage Both: 0.0332246967228
Result: 2.66801936246
Encapsulant discoloration & Diode/J-box problem
Percentage 1

In [18]:
modes_heatmap_all

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.29615,0,2.97335,0.382939,1.45535,2.66802,2.93133,2.48781,1.9492,2.51442
Major delamination,1.29615,NaN,0,1.31385,1.47004,1.48982,1.53284,1.50046,1.2505,0.986978,1.42317
Minor delamination,0,0,NaN,0,0,0,0,0,17.1788,0,0
Backsheet other,2.97335,1.31385,0,NaN,3.21008,2.21152,0,11.059,14.6525,0,0
Internal circuitry discoloration,0.382939,1.47004,0,3.21008,NaN,2.10822,0.368073,1.34761,1.83202,0.0418171,0.359991
Hot spots,1.45535,1.48982,0,2.21152,2.10822,NaN,1.18184,1.52353,1.14502,0,2.06757
Fractured cells,2.66802,1.53284,0,0,0.368073,1.18184,NaN,0,0,0.100027,0
Diode/J-box problem,2.93133,1.50046,0,11.059,1.34761,1.52353,0,NaN,5.94886,0,0
Glass breakage,2.48781,1.2505,17.1788,14.6525,1.83202,1.14502,0,5.94886,NaN,11.0072,0.149381
Permanent soiling,1.9492,0.986978,0,0,0.0418171,0,0.100027,0,11.0072,NaN,1.27477


In [19]:
modes_heatmap_all.iplot(kind='heatmap',colorscale='spectral',
                                filename='modes-heatmap-all', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed before 2000

In [20]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[solar['Begin.Year'] < 2000]

modes_heatmap_pre_2000 = generate_unweighted_heatmap(specified, modes)

Encapsulant discoloration & Major delamination
Percentage 1: 0.24806312769
Percentage 2: 0.759397417504
Percentage Both: 0.202869440459
Result: 1.07692460671
Encapsulant discoloration & Minor delamination
Percentage 1: 0.24806312769
Percentage 2: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.24806312769
Percentage 2: 0.0417503586801
Percentage Both: 0.0417503586801
Result: 4.03123192597
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.24806312769
Percentage 2: 0.396556671449
Percentage Both: 0.0533715925395
Result: 0.542553645608
Encapsulant discoloration & Hot spots
Percentage 1: 0.24806312769
Percentage 2: 0.421520803443
Percentage Both: 0.0890961262554
Result: 0.852074549362
Encapsulant discoloration & Fractured cells
Percentage 1: 0.24806312769
Percentage 2: 0.0274031563845
Percentage Both: 0.0274031563845
Result: 4.03123192597
Encapsulant discoloration & Diode/J-box problem
Percentage 1: 0.24806312769
Percentage 2: 0.120659971306
Perce

In [21]:
modes_heatmap_pre_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='modes-heatmap-pre-2000', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed post 2000

In [22]:
# modes = ['Hot spots', 'Encapsulant discoloration', 'Glass breakage', 'Major delamination', 'Permanent soiling', 'Fractured cells', 'Frame deformation', 'Diode/J-box problem', 'Internal circuitry discoloration', 'Backsheet insulation compromise']
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[solar['Begin.Year'] >= 2000]

modes_heatmap_post_2000 = generate_unweighted_heatmap(specified, modes)

Encapsulant discoloration & Major delamination
Percentage 1: 0.487721021611
Percentage 2: 0.461198428291
Percentage Both: 0.420432220039
Result: 1.86911790414
Encapsulant discoloration & Minor delamination
Percentage 1: 0.487721021611
Percentage 2: 0.00024557956778
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.487721021611
Percentage 2: 0.0120333988212
Percentage Both: 0.0120333988212
Result: 2.05035246727
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.487721021611
Percentage 2: 0.0419941060904
Percentage Both: 0.00147347740668
Result: 0.0719421918341
Encapsulant discoloration & Hot spots
Percentage 1: 0.487721021611
Percentage 2: 0.490667976424
Percentage Both: 0.440815324165
Result: 1.84203337275
Encapsulant discoloration & Fractured cells
Percentage 1: 0.487721021611
Percentage 2: 0.053536345776
Percentage Both: 0.0432220039293
Result: 1.65533043229
Encapsulant discoloration & Diode/J-box problem
Percentage

In [23]:
modes_heatmap_post_2000

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.86912,0,2.05035,0.0719422,1.84203,1.65533,1.79406,0.0788597,0.151878,1.78729
Major delamination,1.86912,NaN,0,0.0442503,2.04147,2.02719,2.14837,0.271033,0.0833948,0.040153,2.08644
Minor delamination,0,0,NaN,0,0,0,0,0,78.3077,0,0
Backsheet other,2.05035,0.0442503,0,NaN,0,1.95485,0,0,0,0,0
Internal circuitry discoloration,0.0719422,2.04147,0,0,NaN,1.90694,4.36933,11.9064,0,1.32294,2.15664
Hot spots,1.84203,2.02719,0,1.95485,1.90694,NaN,2.01934,0,1.56772,0,1.96113
Fractured cells,1.65533,2.14837,0,0,4.36933,2.01934,NaN,0,0,0.345906,0
Diode/J-box problem,1.79406,0.271033,0,0,11.9064,0,0,NaN,9.78846,0,0
Glass breakage,0.0788597,0.0833948,78.3077,0,0,1.56772,0,9.78846,NaN,1.45014,0
Permanent soiling,0.151878,0.040153,0,0,1.32294,0,0.345906,0,1.45014,NaN,0.0569113


In [24]:
modes_heatmap_post_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='modes-heatmap-post-2000', margin=(200,150,120,30))